In [ ]:
import sys
sys.path.append("..")
from figutils import *

In [ ]:
scene_name = 'gradients'
output_dir = os.path.join(OUTPUT_DIR, scene_name)
it = 20

In [ ]:
methods = ["gt", "baseline", 'cv_ps']
method_names = [r"\textbf{(a)} Ground Truth", r"\textbf{(b)} Baseline", r"\textbf{(c)} Ours"]
grads = ["l1", "l2"]
grad_names = [r"$\mathcal{L}^1$", r"$\mathcal{L}^2$"]
gradients = []
imgs = []
scales = [0, 0]
channel = 1

imgs.append(mi.TensorXf(mi.Bitmap(os.path.join(output_dir, "cv_ps", "img_final.exr"))))
imgs.append(mi.TensorXf(mi.Bitmap(os.path.join(output_dir, "cv_ps", "img_F", f"{it//5:04d}.exr"))))
imgs.append(mi.TensorXf(mi.Bitmap(os.path.join(output_dir, "cv_ps", "img", f"{it//5:04d}.exr"))))
ref = mi.TensorXf(mi.Bitmap(os.path.join(output_dir, "img_ref.exr")))

h,w,_ = imgs[0].shape
N = w*h

gradients.append([dr.sign(img - ref) for img in imgs])
gradients.append([(img - ref) for img in imgs])

for i, method in enumerate(methods):
    for j, grad in enumerate(grads):
        scales[j] = dr.maximum(dr.max(dr.abs(gradients[j][i])), scales[j])[0]

scales = [1.0, scales[1]/200]

In [ ]:
sns.set_style('white')

In [ ]:
n_cols = 3
n_rows = 3

crop1 = [0.4, 0.23, 0.1]
crop2 = [0.5, 0.52, 0.12]
crops = [crop1, crop2]
crop_colors = ["r", "g"]

outer_wspace = 0.05
outer_hspace = 0.03

inset_hspace = 0.03
inset_wspace = 0.03

main_aspect = w/h
insets_aspect = gridspec_aspect(2, 2, 1, 1, wspace=inset_wspace, hspace=inset_hspace)
height_ratios = [1/main_aspect, 1/insets_aspect]
outer_aspect = gridspec_aspect(2, 3, 1, height_ratios, wspace=outer_wspace, hspace=outer_hspace)

import matplotlib.gridspec as gridspec
import matplotlib.patches as patches

fig = plt.figure(1, figsize=(TEXT_WIDTH, TEXT_WIDTH / outer_aspect))
outer_gs = fig.add_gridspec(2, 3, wspace=outer_wspace, hspace=outer_hspace, height_ratios=height_ratios)

# scales = [2, 2e-1]

for i, method in enumerate(method_names):
    main_gs = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=outer_gs[0, i], wspace=0.0, hspace=0.0)
    inset_gs = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=outer_gs[1, i], wspace=inset_wspace, hspace=inset_hspace)
    ax = fig.add_subplot(main_gs[0])
    ax.imshow(mi.util.convert_to_bitmap(imgs[i]), interpolation='none')
    disable_ticks(ax)
    if i == 0:
        for l, ((rx, ry, s), color) in enumerate(zip(crops, crop_colors)):
            left = int(rx*w)
            size = int(s*w)
            top = int(ry*h)
            rect = patches.Rectangle((left, top), size, size, linewidth=0.5, edgecolor=color, facecolor='none')
            ax.add_patch(rect)
        ax.set_ylabel("Primal")

    for j, loss_name in enumerate(grads):
        for l, ((rx, ry, s), color) in enumerate(zip(crops, crop_colors)):
            ax = fig.add_subplot(inset_gs[j, l])
            left = int(rx*w)
            size = int(s*w)
            right = left + size
            top = int(ry*h)
            bottom = top + size
            im = ax.imshow(gradients[j][i][top:bottom, left:right, channel], cmap='coolwarm', vmin=-scales[j], vmax=scales[j], interpolation='none')
            disable_ticks(ax)
            plt.setp(ax.spines.values(), color=color)
            rect = patches.Rectangle((0, 0), size-1, size-1, linewidth=1.0, edgecolor=color, facecolor='none')
            ax.add_patch(rect)

            if i == 0 and l == 0:
                ax.set_ylabel(grad_names[j])
            if i == 2 and l == 1:
                cbax = ax.inset_axes([1.04, 0.0, 0.08, 1.0], transform=ax.transAxes)
                cbar = fig.colorbar(im, cax=cbax, ticks=[-scales[j]*0.9, 0, scales[j]*0.9], format="%.1f")
                cbar.outline.set_visible(False)
                cbar.ax.tick_params(size=0)
                

    # Ghost axes for the labels (https://stackoverflow.com/a/69117807)
    ax_label = fig.add_subplot(outer_gs[1, i])
    ax_label.axis('off')
    ax_label.set_title(method)

save_fig("gradients", pad_inches=0.02)